<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 3.2.1
# *Querying the International Space Station*

## The OpenNotify API

The OpenNotify API exposes a few attributes of the International Space Station (ISS) via a simple, authentication-free interface. The simplicity of this API precludes any need for a dedicated Python library. However, as with many APIs, it accepts requests according to HTTP standards and returns responses in JSON format, so the Python libraries request and json will make managing the I/O simpler still.

In [1]:
import requests
import json
from datetime import datetime, date, time

This request fetches the latest position of the international space station:

In [2]:
response = requests.get("http://api.open-notify.org/iss-now.json")

Print the status code and text of the response:

In [4]:
#ANSWER
response.status_code

200

In [5]:
#ANSWER
response.text

'{"iss_position": {"latitude": "0.6000", "longitude": "-107.3632"}, "message": "success", "timestamp": 1720744729}'

We can use another API to request the current position of the ISS and the next few times at which it will be over a certain location. The latitude and longitude of Sydney are (-33.87, 151.21).

In [34]:
response = requests.get("https://api.g7vrd.co.uk/v1/satellite-passes/25544/-33.87/151.21.json?minelevation=0&hours=24")

Print the response header:

In [7]:
#ANSWER
response.headers

{'Date': 'Fri, 12 Jul 2024 00:45:28 GMT', 'Server': 'Apache', 'Vary': 'Origin,Access-Control-Request-Method,Access-Control-Request-Headers', 'Access-Control-Allow-Origin': '*', 'X-Content-Type-Options': 'nosniff', 'X-XSS-Protection': '0', 'Cache-Control': 'no-cache, no-store, max-age=0, must-revalidate', 'Pragma': 'no-cache', 'Expires': '0', 'X-Frame-Options': 'DENY', 'Content-Type': 'application/json', 'Keep-Alive': 'timeout=5, max=100', 'Connection': 'Keep-Alive', 'Transfer-Encoding': 'chunked'}

Print the content of the response (the data that the server returned):

In [8]:
#ANSWER
response.content

b'{"api_status":"ALPHA","request_timestamp":"2024-07-12T00:45:28.432429023Z","norad_id":25544,"satellite_name":"ISS","tle_last_retrieved":"2024-07-11T10:06:09.045762639Z","lat":-33.87,"lon":151.21,"hours":24,"min_elevation":0,"query_ms":14,"passes":[{"start":"2024-07-12T01:46:48.418Z","tca":"2024-07-12T01:49:48.418Z","end":"2024-07-12T01:52:53.418Z","aos_azimuth":207,"los_azimuth":140,"max_elevation":3.0},{"start":"2024-07-12T03:23:28.418Z","tca":"2024-07-12T03:27:58.418Z","end":"2024-07-12T03:32:43.418Z","aos_azimuth":210,"los_azimuth":96,"max_elevation":11.0},{"start":"2024-07-12T04:59:48.418Z","tca":"2024-07-12T05:05:18.418Z","end":"2024-07-12T05:10:53.418Z","aos_azimuth":225,"los_azimuth":48,"max_elevation":78.0},{"start":"2024-07-12T06:37:38.418Z","tca":"2024-07-12T06:41:38.418Z","end":"2024-07-12T06:45:33.418Z","aos_azimuth":257,"los_azimuth":350,"max_elevation":7.0},{"start":"2024-07-12T20:03:48.418Z","tca":"2024-07-12T20:08:48.418Z","end":"2024-07-12T20:13:38.418Z","aos_azimuth

Note that this is a Python byte string:

In [9]:
print(type(response.content))

<class 'bytes'>


Print just the "content-type" value from the header:

In [12]:
#ANSWER
response.headers['content-type']

'application/json'

JSON was designed to be easy for computers to read, not for people. The `requests` library can decode the JSON byte string:

In [11]:
overheads = response.json()
print(overheads)

{'api_status': 'ALPHA', 'request_timestamp': '2024-07-12T00:45:28.432429023Z', 'norad_id': 25544, 'satellite_name': 'ISS', 'tle_last_retrieved': '2024-07-11T10:06:09.045762639Z', 'lat': -33.87, 'lon': 151.21, 'hours': 24, 'min_elevation': 0, 'query_ms': 14, 'passes': [{'start': '2024-07-12T01:46:48.418Z', 'tca': '2024-07-12T01:49:48.418Z', 'end': '2024-07-12T01:52:53.418Z', 'aos_azimuth': 207, 'los_azimuth': 140, 'max_elevation': 3.0}, {'start': '2024-07-12T03:23:28.418Z', 'tca': '2024-07-12T03:27:58.418Z', 'end': '2024-07-12T03:32:43.418Z', 'aos_azimuth': 210, 'los_azimuth': 96, 'max_elevation': 11.0}, {'start': '2024-07-12T04:59:48.418Z', 'tca': '2024-07-12T05:05:18.418Z', 'end': '2024-07-12T05:10:53.418Z', 'aos_azimuth': 225, 'los_azimuth': 48, 'max_elevation': 78.0}, {'start': '2024-07-12T06:37:38.418Z', 'tca': '2024-07-12T06:41:38.418Z', 'end': '2024-07-12T06:45:33.418Z', 'aos_azimuth': 257, 'los_azimuth': 350, 'max_elevation': 7.0}, {'start': '2024-07-12T20:03:48.418Z', 'tca': '2

What kind of object did this give us?

In [14]:
#ANSWER:
type(overheads)

dict

Python dicts are easier to work with, but the data we want is still buried in that data structure, so we have to dig it out. First, extract the `passes` value to a separate list:

In [17]:
#ANSWER:
passes=overheads['passes']
passes

[{'start': '2024-07-12T01:46:48.418Z',
  'tca': '2024-07-12T01:49:48.418Z',
  'end': '2024-07-12T01:52:53.418Z',
  'aos_azimuth': 207,
  'los_azimuth': 140,
  'max_elevation': 3.0},
 {'start': '2024-07-12T03:23:28.418Z',
  'tca': '2024-07-12T03:27:58.418Z',
  'end': '2024-07-12T03:32:43.418Z',
  'aos_azimuth': 210,
  'los_azimuth': 96,
  'max_elevation': 11.0},
 {'start': '2024-07-12T04:59:48.418Z',
  'tca': '2024-07-12T05:05:18.418Z',
  'end': '2024-07-12T05:10:53.418Z',
  'aos_azimuth': 225,
  'los_azimuth': 48,
  'max_elevation': 78.0},
 {'start': '2024-07-12T06:37:38.418Z',
  'tca': '2024-07-12T06:41:38.418Z',
  'end': '2024-07-12T06:45:33.418Z',
  'aos_azimuth': 257,
  'los_azimuth': 350,
  'max_elevation': 7.0},
 {'start': '2024-07-12T20:03:48.418Z',
  'tca': '2024-07-12T20:08:48.418Z',
  'end': '2024-07-12T20:13:38.418Z',
  'aos_azimuth': 349,
  'los_azimuth': 116,
  'max_elevation': 17.0},
 {'start': '2024-07-12T21:39:48.418Z',
  'tca': '2024-07-12T21:45:18.418Z',
  'end': '202

Now extract the `start` strings into an array called `srisetimes`:

In [20]:
#ANSWER:
srisetimes= [xpass['start'] for xpass in passes]
srisetimes

['2024-07-12T01:46:48.418Z',
 '2024-07-12T03:23:28.418Z',
 '2024-07-12T04:59:48.418Z',
 '2024-07-12T06:37:38.418Z',
 '2024-07-12T20:03:48.418Z',
 '2024-07-12T21:39:48.418Z',
 '2024-07-12T23:18:43.418Z',
 '2024-07-13T00:58:18.418Z']

These are strings. We convert these to an array of Python `datetime` values called `risetimes`:

In [21]:
risetimes = [datetime.strptime(xpass['start'], "%Y-%m-%dT%H:%M:%S.%fZ") for xpass in passes]
risetimes

[datetime.datetime(2024, 7, 12, 1, 46, 48, 418000),
 datetime.datetime(2024, 7, 12, 3, 23, 28, 418000),
 datetime.datetime(2024, 7, 12, 4, 59, 48, 418000),
 datetime.datetime(2024, 7, 12, 6, 37, 38, 418000),
 datetime.datetime(2024, 7, 12, 20, 3, 48, 418000),
 datetime.datetime(2024, 7, 12, 21, 39, 48, 418000),
 datetime.datetime(2024, 7, 12, 23, 18, 43, 418000),
 datetime.datetime(2024, 7, 13, 0, 58, 18, 418000)]

Finally, use `risetime.strftime` to print these in a format that people understand:

```
e.g.
18/10/22 07:05
18/10/22 08:41
18/10/22 10:20
18/10/22 12:00
18/10/22 01:37
18/10/22 03:13
```



In [29]:
#ANSWER:
for times in risetimes:
    print(datetime.strftime(times,'%d/%m/%y %H:%M'))


12/07/24 01:46
12/07/24 03:23
12/07/24 04:59
12/07/24 06:37
12/07/24 20:03
12/07/24 21:39
12/07/24 23:18
13/07/24 00:58


Finally, here is an endpoint that tells us who is on board:

In [35]:
response = requests.get("http://api.open-notify.org/astros.json")

Referring to the methods used above, extract the number of astronauts and their names:

In [39]:
#ANSWER:
astronauts= response.json()
print("Number of Astronauts: ",astronauts['number']) 
for astro in astronauts['people']:
        print(astro["name"])

Number of Astronauts:  12
Oleg Kononenko
Nikolai Chub
Tracy Caldwell Dyson
Matthew Dominick
Michael Barratt
Jeanette Epps
Alexander Grebenkin
Butch Wilmore
Sunita Williams
Li Guangsu
Li Cong
Ye Guangfu


## HOMEWORK


1. Write a simple handler for the response status code (refer to lab resources slide for HTTP response codes). As this Jupyter Notebook is an interactive device, the handler does not need to manage subsequent code execution (i.e. by branching or aborting execution), although it should return something that could be used to do so if deployed in a Python program.

In [42]:
#ANSWER:
def handleResponse(response, verbose = False):
    if response.status_code == 200:
        return False, response.status_code
    else:
        return True, response.status_code
    '''
    Returns Boolean Value, Status Code,
    '''
  # if Status Code is 200 return false, and status code
  # Otherwise Return True and Status Code

2. Test your response handler on some correct and incorrect API calls.

In [43]:
response = requests.get("http://api.open-notify.org/astros.json")
if handleResponse(response)[0]:
    print('API call failed. Resolve issue before continuing!')

response = requests.get("http://api.open-notify.org/iss-now.json")
handleResponse(response, True)[0]

False

>

>

>



---



---



> > > > > > > > > © 2024 Institute of Data


---



---



